Package imports

In [ ]:
#r "nuget: Neo4j.Driver, 5.17"
#r "nuget: Microsoft.SemanticKernel, 1.3"
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets"

using Neo4j.Driver;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ComponentModel;
using System.Text.Json;

load secrets into config

In [ ]:
var config = new ConfigurationBuilder()
    .AddUserSecrets("2a78a9a7-426d-457c-b4d0-42438d70fa71")
    .Build();

create and validate driver config

In [ ]:
var driver = GraphDatabase.Driver("bolt://localhost:7687", AuthTokens.Basic("neo4j", config["neo4j:password"]));
await driver.VerifyConnectivityAsync();

Create kernel functions

In [ ]:
var kernelFunction = KernelFunctionFactory.CreateFromMethod(
    async (string cypherQuery, bool isWriteQuery) => 
    { 
        var result = await driver.ExecutableQuery(cypherQuery).WithConfig(new QueryConfig(isWriteQuery ? RoutingControl.Writers : RoutingControl.Readers, "neo4j")).ExecuteAsync();
        return JsonSerializer.Serialize(result);
    }, 
    "queryDatabase",
    "run a query on neo4j database with cypher query",
    new KernelParameterMetadata[] { new("cypherQuery"), new("isWriteQuery") });

Build a kernel injecting kernel functions as a plugin

In [ ]:
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion("gpt-3.5-turbo", config["OAI:ApiKey"]);
builder.Plugins.AddFromFunctions("neo4jdriver", new [] {kernelFunction});
var kernel = builder.Build();

create chat service

In [ ]:
var chatService = kernel.GetRequiredService<IChatCompletionService>();
var promptSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};
var chatHistory = new ChatHistory();

async Task Chat(string userInput)
{
    if (string.IsNullOrEmpty(userInput))
        return;

    chatHistory.AddUserMessage(userInput);
    var response = await chatService.GetChatMessageContentAsync(chatHistory, promptSettings, kernel);
    chatHistory.AddMessage(response.Role, response.Content ?? string.Empty);
    Console.WriteLine(response);
}

call the AI

In [ ]:
await Chat("How many nodes are in the database?");

In [ ]:
await Chat("What is the schema of the database?");

In [ ]:
await Chat("Create a person node with name = 'Andreas'");